# Figures for The atlas of urban communities

### Packages

### Data

### Figure 1 - network layout, bubble maps

In [ ]:
# Atlas of urban communities

def degree_bubble_map(city, mob_cnt_df, follow_hh):
    """
    For a given city draws the bubble networks on a map of 
    the mobility (communitng) and
    the socail ties (follower home-home).
    
    ex: degree_bubble_map('boston', mobility, follow_hh)
    
    """
    
    # Create edge lines
    # geometry = edges between 
    # - home and work centroids
    # - homes of follower pairs tracts' centroids
    
    # filter for the city
    mob_cnt_df = deepcopy(mob_cnt_df[mob_cnt_df['city'] == city])
    follow_hh = deepcopy(follow_hh[follow_hh['city'] == city])
    
    # create contour
    follow_hh = follow_hh.set_geometry(follow_hh['polygon_home'])
    follow_hh['buffered_tract'] = follow_hh.buffer(0.01)
    follow_hh = follow_hh.set_geometry(follow_hh['buffered_tract'])
    follow_hh['dissolve'] = 1
    city_boundary = follow_hh.dissolve(by = 'dissolve')
    type(city_boundary)
  
    mob_cnt_df['geometry'] = mob_cnt_df.apply(lambda row: line(row['centroid_home'],row['centroid_work']), axis = 1) # KERDES MIERT KELL? ,axis=1)
    follow_hh['geometry'] = follow_hh.apply(lambda row: line(row['centroid_home'],row['centroid_home_1']), axis = 1) #,axis=1) 

    # convert into GeoDataFrame
    gdf_mobility= GeoDataFrame(mob_cnt_df, crs="EPSG:4326") # edges
    gdf_follow_hh = GeoDataFrame(follow_hh, crs="EPSG:4326")

    # GeoDataFrames for the bubbles
    gdf_mobility_nodes = gdf_mobility.set_geometry('centroid_work')
    gdf_follow_hh_nodes = gdf_follow_hh.set_geometry('centroid_home')

    
    # Draw figure
    f, ax = plt.subplots(1,2, figsize=(16,8))
    f.suptitle(city, fontsize = 20)
    
    # mobility
    ax[0].set_title('Commuting', size = 16)
    gdf_mobility.plot(ax = ax[0], linewidth=gdf_mobility['cnt']*0.05, color = color_mob) # edges
    gdf_mobility_nodes.plot(ax = ax[0], alpha=0.5, markersize=gdf_mobility_nodes['cnt']*2, color='black')
    city_boundary.boundary.plot(ax = ax[0], color = 'black')
    ax[0].set_axis_off()

    # fol_hh
    ax[1].set_title('Social ties', size = 16)
    gdf_follow_hh.plot(ax = ax[1], alpha = 0.7, linewidth = gdf_follow_hh['cnt']*0.01, color = color_fol_hh)  #'#33a02c')#'#01665e')#'#1b7837')#'#542788')# #########'#762a83')# #    
    gdf_follow_hh_nodes.plot(ax = ax[1], alpha=0.2, markersize = gdf_follow_hh_nodes['all_home_cnt'].astype('float')*2, color='black')
    city_boundary.boundary.plot(ax = ax[1], color = 'black')
    ax[1].set_axis_off()
    
    plt.show()
    filename = 'bubble_maps_' + city
    # savefig(filename, dpi=1200, bbox_inches='tight')
    #plt.savefig("../figures/" + filename  + ".png", density=300, bbox_inches='tight')

    # TODO CHECK AND CORRECT SIZES

### Figure 2 - maps colored with NG and Spa communities

In [ ]:
# Atlas_of_urban_com
# depreciated comments cut

# Consensus community map drawer
def community_map(city, g_type, tract_df):
    """
    For a given network and algorithm used for calculating communities,
    it draws the colored community map eith tract boundaries.
    
    It uses the previously loaded `tract_geoms` GeoDataFrame in which geometrical data is stored,
    the `tract_df` pandas.DataFrames, in which consensus clustering labels are stored
    and colors_fol_hh_mgn color list.
    
    ex: community_map('boston','fol_hh', tract_df)
    
    Parameters:
    -----------
    city : str
        name of the city, lowercase, '_' instead of ' '
        
    g_type : str
        either "mob" as mobility or "fol_hh" as follow_hh 
        selects the type of graph to return
        
    tract_df : pd.DataFrame
        contains community labels for each tract
        
    Returns:
    --------
    
    plot maps and save it to figures
        
    """
    
    # filter data to the given network's given communities
    map_df = deepcopy(tract_df[(tract_df['city'] == city) & (tract_df['g_type'] == g_type)])
    # merge to geodata
    map_tracts = pd.merge(tract_geoms[['geometry', 'full_geoid', 'centroid']], map_df, left_on='full_geoid', right_on='geoid')      
    
    f, ax = plt.subplots(1,2, figsize=(16,11))
    #f.suptitle(city + ' '+ g_type, fontsize = 20)
    # JAVITANI
    #f.suptitle('Communities based on socail ties in Boston', fontsize = 20) ### JAVÍTANI!!!!
    
    # Newman-Girwan
    ax[0].set_title('Newman-Girvan', size = 16)
    map_tracts_mgn = deepcopy(map_tracts[map_tracts['algorithm_type'] == 'mgn'])
    cmap = matplotlib.colors.ListedColormap(colors_fol_hh_mgn_1)
    map_tracts_mgn.plot(ax=ax[0], column = 'S_cons', cmap=cmap, legend=True)
    map_tracts.boundary.plot(ax=ax[0], color='black', linewidth=0.015)
    #map_tracts.centroid.plot(ax=ax, color='red', markersize=0.5)
    ax[0].set_axis_off()
    
    # Spatial a la Expert
    ax[1].set_title('Expert', size = 16)
    map_tracts_ms = deepcopy(map_tracts[map_tracts['algorithm_type'] == 'ms'])
    # SAME cmap = matplotlib.colors.ListedColormap(colors_fol_hh_ms)
    map_tracts_ms.plot(ax=ax[1], column = 'S_cons', cmap=cmap, legend=True)
    map_tracts.boundary.plot(ax=ax[1], color='black', linewidth=0.015)
    #map_tracts.centroid.plot(ax=ax, color='red', markersize=0.5)
    ax[1].set_axis_off()
    
    
    f.delaxes(f.axes[2]) # remove colorbar
    f.delaxes(f.axes[2]) # from the second part of figure too
    
    plt.tight_layout()
    plt.show()
    #plt.savefig("../figures/example_community_maps.png", density=300, bbox_inches='tight')

### Figure 3 - community quality & demography scatterplots made from communities

In [ ]:
# Stats_of_urban_com

def community_scatter(demo):
    
    # filter out small communities
    community_df_filt = community_df[community_df['tract_sum'] > 10]
    
    community_df_ms_fol_hh = deepcopy(community_df_filt[(community_df_filt['algorithm_type'] == 'ms') & (community_df_filt['g_type'] == 'fol_hh')])
    community_df_mgn_fol_hh = deepcopy(community_df_filt[(community_df_filt['algorithm_type'] == 'mgn') & (community_df_filt['g_type'] == 'fol_hh')])
    
    fig, ax = plt.subplots(1,2, figsize = (14,7))#, sharey = True) # 0604
    #fig.suptitle('Correlation of communities and their incomes', fontsize=20)
    
    x_mgn = community_df_mgn_fol_hh['mod_S_prop']
    y_mgn = community_df_mgn_fol_hh[demo]
    w_mgn = community_df_mgn_fol_hh['tract_sum']
    
    # filt[er out nan-s for the slope calculations - KERDES ez igy jo?
    mask_mgn = ~np.isnan(x_mgn) & ~np.isnan(y_mgn)
        
    x_mgn = community_df_mgn_fol_hh['mod_S_prop'][mask_mgn]
    y_mgn = community_df_mgn_fol_hh[demo][mask_mgn]
    w_mgn = community_df_mgn_fol_hh['tract_sum'][mask_mgn]
   
    model_mgn = sklearn.linear_model.LinearRegression()
    
    xx_mgn = np.array([x_mgn]).T
    yy_mgn = np.array([y_mgn]).T
    ww_mgn = np.array(w_mgn).T
    
    model_mgn.fit(xx_mgn, yy_mgn, sample_weight = ww_mgn)
    line_x_mgn = np.array([np.linspace(min(x_mgn), max(x_mgn), 100)]).T
    pred_mgn = model_mgn.predict(line_x_mgn)
    y_pred_mgn = model_mgn.predict(xx_mgn)
    r_mgn = sklearn.metrics.r2_score(yy_mgn, y_pred_mgn, sample_weight = ww_mgn)
    intercept_mgn = model_mgn.intercept_[0]
    slope_mgn = model_mgn.coef_[0][0]
    line_mgn = f'Regression line: y={intercept_mgn:.2f}+{slope_mgn:.2f}x, r={r_mgn:.2f}'
    
    ax[0].plot(line_x_mgn, pred_mgn)

    ax[0].scatter(x_mgn, y_mgn, c = 'purple', alpha = 0.8, label = 'Social ties Newman-Girvan')
    
    ax[0].plot(x_mgn, intercept_mgn + slope_mgn * x_mgn, c = 'purple', label=line_mgn)
    
    ax[0].set_title('Newman-Girvan', size = 14)
   
    
    ax[0].set_xlabel('Modularity contribution', size = 14)
    ax[0].set_ylabel(demo, size = 14)
    ax[0].xaxis.set_major_formatter(mtick.PercentFormatter(1)) # % format
    ax[0].grid()
    ax[0].legend()   
    
    
    # EXPERT
    
    x_ms = community_df_ms_fol_hh['mod_S_prop']
    y_ms = community_df_ms_fol_hh[demo]
    w_ms = community_df_ms_fol_hh['tract_sum']    
    
    # filt[er out nan-s for the slope calculations - KERDES ez igy jo?
    mask_ms = ~np.isnan(x_ms) & ~np.isnan(y_ms)
    
    x_ms = community_df_ms_fol_hh['mod_S_prop'][mask_ms]
    y_ms = community_df_ms_fol_hh[demo][mask_ms]
    w_ms = community_df_ms_fol_hh['tract_sum'][mask_ms]
    
    xx_ms = np.array([x_ms]).T
    yy_ms = np.array([y_ms]).T
    ww_ms = np.array(w_ms).T
    
    model_ms = sklearn.linear_model.LinearRegression()    
    model_ms.fit(xx_ms, yy_ms, sample_weight = ww_ms)

    line_x_ms = np.array([np.linspace(min(x_ms), max(x_ms), 100)]).T
    pred_ms = model_ms.predict(line_x_ms)
    
    y_pred_ms = model_ms.predict(xx_ms)
    
    r_ms = sklearn.metrics.r2_score(yy_ms, y_pred_ms, sample_weight = ww_ms)
    
    
    intercept_ms = model_ms.intercept_[0]
    slope_ms = model_ms.coef_[0][0]
    
    line_ms = f'Regression line: y={intercept_ms:.2f}+{slope_ms:.2f}x, r={r_ms:.2f}' # TODO megnezni, hogy mi az az f-string
    
    ax[1].plot(line_x_ms, pred_ms)
    ax[1].scatter(x_ms, y_ms, c = 'green', alpha = 0.8, label = 'Social ties Expert')
    ax[1].plot(x_ms, intercept_ms + slope_ms * x_ms, c = 'green', label=line_ms)
    
    # TODO color with city
    
    
    ax[1].set_title('Expert', size = 14)
    
    y_min = min(min(y_mgn) * 0.9 - 0.00001, min(y_ms) * 0.9 - 0.00001) # y axis shared
    y_max = max(max(y_mgn)*1.1, max(y_ms)*1.1)
    ax[0].set_ylim(y_min, y_max)
    ax[1].set_ylim(y_min, y_max)
    ax[1].set_ylabel(demo, size = 14)
       
    ax[1].set_xlabel('Modularity contribution', size = 14)
    ax[1].xaxis.set_major_formatter(mtick.PercentFormatter(1.0)) # % format
        
    ax[1].grid()
    ax[1].legend()

## Figure 4

## Figure 5